<h1><center> Unauthorised Activity Detection for Bank Lockers</center></h1>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/project"

In [7]:
%cd /content/drive/MyDrive/project/

/content/drive/MyDrive/project


In [ ]:
!kaggle datasets download -d ankan1998/weapon-detection-dataset ##Dataset link :- https://www.kaggle.com/datasets/ankan1998/weapon-detection-dataset

In [ ]:
!unzip /content/drive/MyDrive/project/weapon-detection-dataset.zip

In [ ]:
# /content/drive/MyDrive/project/Sohas_weapon-Detection

In [ ]:
!pip install tensorflow==2.7.0
!pip install opencv-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.7/489.7 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.1/463.1 KB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.2 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.11.0
    Uninstalling tensorflow-estimator-2.11.0:
      Successfully uninstalled tensorflow-estimator-2.11.0
  Attempting uninstall: keras
    Found existing installation: keras 2.11.0
    Uninstalling keras-2.11.0:
      Successfully uninstalled keras-2.11.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 23.3.3
    Uninstalling flatbuffers-23.3.3:
      Successfully uninstalled flatbuffers-23.3.3
  Attempting uninstall: tensorflow
    Foun

In [14]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential

# Define the CNN model for feature extraction
def cnn_model(input_shape=(224, 224, 3)):
    model = Sequential()
    model.add(Conv2D(64, kernel_size=(3,3), activation='relu', input_shape=input_shape, padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    
    model.add(Conv2D(128, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    
    model.add(Conv2D(256, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    
    model.add(Conv2D(512, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    
    model.add(Conv2D(512, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    
    model.add(Flatten())
    
    return model

# Define the RNN model for temporal processing
def rnn_model(timesteps, num_classes):
    model = Sequential()
    #model.add(TimeDistributed(Conv2D(64, (3,3), activation='relu'), input_shape=(timesteps, 224, 224, 3)))
    #model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(256, input_shape=(timesteps, 512), return_sequences=True))
    model.add(LSTM(256))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    
    return model

# Define the combined CNN-RNN model
def cnn_rnn_model(input_shape, timesteps, num_classes):
    input_layer = tf.keras.layers.Input(shape=(timesteps,) + input_shape)
    cnn = cnn_model(input_shape)
    cnn_outputs = TimeDistributed(cnn)(input_layer)
    rnn = rnn_model(timesteps, num_classes)
    rnn_outputs = rnn(cnn_outputs)
    
    model = tf.keras.Model(inputs=input_layer, outputs=rnn_outputs)
    
    return model


In [15]:
import os
import cv2
import numpy as np
import xml.etree.ElementTree as ET

dataset_dir = "/content/drive/MyDrive/project/Sohas_weapon-Detection/images"
path = "/content/drive/MyDrive/project/Sohas_weapon-Detection/annotations/xmls"
classes={'pistol': 0, 'rifle': 1, 'submachinegun': 2, 'assaultrifle': 3, 'rocketlauncher': 4, 'grenade': 5, 'knife': 6, 'none': 7}

def load_data(dataset_dir, path, img_size=(224,224)):
    X = []
    y=[]
    for file in os.listdir(dataset_dir):
        img_path = os.path.join(dataset_dir, file)
        img = cv2.imread(img_path)
        img = cv2.resize(img, img_size)
        X.append(img)
    #path = "/content/drive/MyDrive/project/Sohas_weapon-Detection/annotations/xmls"
    #classes={'pistol': 0, 'rifle': 1, 'submachinegun': 2, 'assaultrifle': 3, 'rocketlauncher': 4, 'grenade': 5, 'knife': 6, 'none': 7}
    #y=[]
    #for file in os.listdir(path):
        if file.endswith(".jpg"):
            xml_file=os.path.join(path, file.replace(".jpg", ".xml"))
        elif file.endswith(".JPG"):
            xml_file=os.path.join(path, file.replace(".JPG", ".xml"))
        elif file.endswith(".jpeg"):
            xml_file=os.path.join(path, file.replace(".jpeg", ".xml"))
        else:
            xml_file=os.path.join(path, file.replace(".JPEG", ".xml"))
        tree = ET.parse(xml_file)
        root = tree.getroot()

        # Extract image height
        #height = float(root.find("size/height").text)
        #width = float(root.find("size/width").text)

        # Extract bounding box coordinates for object named "knife"
        
        name=str(root.find("object/name").text)
        if name in classes:
          class_num=classes[name]
        else:
          class_num=7
          
        #xmin = float(root.find("object/bndbox/xmin").text)/width
        #ymin = float(root.find("object/bndbox/ymin").text)/height
        #xmax = float(root.find("object/bndbox/xmax").text)/width
        #ymax = float(root.find("object/bndbox/ymax").text)/height
        #h = ymax - ymin  # Height of bounding box
        #w = xmax - xmin  # width
        #x_centre=(xmin+xmax)/2
        #y_centre=(ymin+ymax)/2
        #class_num=float(class_num)
        temp=[0]*8
        temp[class_num]=1;
        #y.append([[class_num, x_centre, y_centre, w, h]])
        y.append(temp)
            
    '''label_file = open(label_path, "r")
    labels = label_file.readlines()
    label_file.close()
    labels = [x.strip().split(" ") for x in labels]'''
    X = np.array(X)
    y = np.array(y, dtype=float)
    return X, y



In [ ]:
# Define the label encoder for converting class names to integers
'''class_names = ['pistol', 'rifle', 'submachinegun', 'assaultrifle', 'rocketlauncher', 'grenade', 'none']
class_dict = {class_names[i]: i for i in range(len(class_names))}
num_classes = len(class_names)

def label_encoder(labels):
    encoded_labels = np.zeros((len(labels), len(labels[0]), num_classes))
    for i in range(len(labels)):
        for j in range(len(labels[i])):
            if labels[i][j][0] in class_dict:
                class_index = class_dict[labels[i][j][0]]
                encoded_labels[i][j][class_index] = 1
    return encoded_labels
'''
# Load the dataset
dataset_dir = "/content/drive/MyDrive/project/Sohas_weapon-Detection/images"
path = "/content/drive/MyDrive/project/Sohas_weapon-Detection/annotations/xmls"
X_train, y_train = load_data(dataset_dir, path)

'''dataset_dir = "/content/drive/MyDrive/project/Sohas_weapon-Detection/images_test"
path = "/content/drive/MyDrive/project/Sohas_weapon-Detection/annotations_test/xmls"
X_test, y_test = load_data(dataset_dir, path)
'''
# Preprocess the dataset
X_train = X_train / 255.0
#X_test=X_test/255.0
#y = label_encoder(y)

# Split the dataset into train and test sets
'''split_index = int(0.8 * len(X))
X_train, y_train = X[:split_index], y[:split_index]
X_test, y_test = X[split_index:], y[split_index:]
X.shape
y.shape
X_train.shape'''


In [17]:
dataset_dir = "/content/drive/MyDrive/project/Sohas_weapon-Detection/images_test"
path = "/content/drive/MyDrive/project/Sohas_weapon-Detection/annotations_test/xmls"
X_test, y_test = load_data(dataset_dir, path)

X_test = X_test/255.0

In [ ]:
X_train.shape

(1000, 224, 224, 3)

In [ ]:
y_train.shape

(1000, 8)

In [ ]:
X_test.shape

(472, 224, 224, 3)

In [ ]:
y_test.shape

(472, 8)

In [ ]:
# Define the model and compile it

model = cnn_rnn_model(input_shape=X_train.shape[1:], timesteps=1, num_classes=8)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

X_train1 = np.reshape(X_train, (-1, 1, 224, 224, 3))
X_test1 = np.reshape(X_test, (-1, 1, 224, 224, 3))
# Train the model
model.fit(X_train1, y_train, batch_size=16, epochs=20, validation_data=(X_test1, y_test))


Epoch 1/20
63/63 [==============================] - 29s 164ms/step - loss: 0.3348 - accuracy: 0.9150 - val_loss: 1.6614 - val_accuracy: 0.3559
Epoch 2/20
63/63 [==============================] - 7s 111ms/step - loss: 0.1094 - accuracy: 0.9630 - val_loss: 1.4508 - val_accuracy: 0.5148
Epoch 3/20
63/63 [==============================] - 7s 112ms/step - loss: 0.1353 - accuracy: 0.9600 - val_loss: 2.1692 - val_accuracy: 0.3644
Epoch 4/20
63/63 [==============================] - 7s 113ms/step - loss: 0.0582 - accuracy: 0.9840 - val_loss: 3.3837 - val_accuracy: 0.2945
Epoch 5/20
63/63 [==============================] - 7s 114ms/step - loss: 0.0537 - accuracy: 0.9860 - val_loss: 2.8823 - val_accuracy: 0.3136
Epoch 6/20
63/63 [==============================] - 7s 113ms/step - loss: 0.0964 - accuracy: 0.9670 - val_loss: 3.1854 - val_accuracy: 0.3750
Epoch 7/20
63/63 [==============================] - 7s 114ms/step - loss: 0.0532 - accuracy: 0.9840 - val_loss: 1.9137 - val_accuracy: 0.4852
Epoch

In [ ]:
y_train

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [ ]:
y_test

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [ ]:
model.save("/content/drive/MyDrive/project/model.h5")

In [3]:
import tensorflow as tf
model = tf.keras.models.load_model("/content/drive/MyDrive/project/model.h5")
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 224, 224, 3)]  0         
                                                                 
 time_distributed (TimeDistr  (None, 1, 25088)         3916672   
 ibuted)                                                         
                                                                 
 sequential_1 (Sequential)   (None, 8)                 26480648  
                                                                 
Total params: 30,397,320
Trainable params: 30,394,376
Non-trainable params: 2,944
_________________________________________________________________


In [20]:
import tensorflow as tf
import cv2
import numpy as np
class_names=['pistol', 'rifle', 'submachinegun', 'assaultrifle', 'rocketlauncher', 'grenade', 'none', 'knife']
# Load the trained model
model = tf.keras.models.load_model("/content/drive/MyDrive/project/model.h5")

# Open the video file
cap = cv2.VideoCapture("/content/drive/MyDrive/project/knife.mp4")
#knife.mp4,atm.mp4
#cap = cv2.VideoCapture("/content/drive/MyDrive/project/man_walking.mp4")
#cap = cv2.VideoCapture("/content/drive/MyDrive/project/bank.mp4")

# Define the object detection function
def detect_objects(frame):
    # Resize the frame and preprocess it
    frame = cv2.resize(frame, (224,224))
    frame = np.expand_dims(frame, axis=0)
    frame = frame / 255.0
    
    # Use the model to predict the class probabilities for each frame
    predictions = model.predict(np.array([frame]))
    print(predictions)
    
    # Convert the probabilities to class labels
    labels = []
    for i in range(predictions.shape[1]):
        class_index = np.argmax(predictions[0])
        class_name = class_names[class_index]
        labels.append(class_name)
    #print(labels)
    '''
    # Draw bounding boxes around the detected objects
    for i in range(predictions.shape[1]):
        if labels[i] != 'none':
            top, left, bottom, right = [int(x) for x in predictions[0][i][:-1]]
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
            cv2.putText(frame, labels[i], (left, top-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
    '''
    print(labels)
    return labels[0],frame

# Process each frame of the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    prediction,frame = detect_objects(frame)
    #cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

#print("\n\nObject Predicted: ":prediction)


1/1 [==============================] - 9s 9s/step
[[5.0669227e-05 9.7996373e-05 6.9845220e-05 6.5447406e-05 6.8816946e-05
  7.6897108e-05 3.5824772e-02 9.6374553e-01]]
['knife', 'knife', 'knife', 'knife', 'knife', 'knife', 'knife', 'knife']
1/1 [==============================] - 0s 27ms/step
[[6.0733382e-05 1.1857533e-04 8.5397485e-05 8.0793056e-05 8.2125247e-05
  9.2845759e-05 4.8162509e-02 9.5131695e-01]]
['knife', 'knife', 'knife', 'knife', 'knife', 'knife', 'knife', 'knife']
1/1 [==============================] - 0s 33ms/step
[[5.7052399e-05 1.1042285e-04 8.0132253e-05 7.5051779e-05 7.7537778e-05
  8.7302033e-05 4.2738777e-02 9.5677382e-01]]
['knife', 'knife', 'knife', 'knife', 'knife', 'knife', 'knife', 'knife']
1/1 [==============================] - 0s 26ms/step
[[6.21354848e-05 1.21596575e-04 8.76340273e-05 8.27946860e-05
  8.40831126e-05 9.50205722e-05 4.92826663e-02 9.50184107e-01]]
['knife', 'knife', 'knife', 'knife', 'knife', 'knife', 'knife', 'knife']
1/1 [=================

In [21]:
print("Object Predicted: "+ prediction)

Object Predicted: knife
